In [54]:
import numpy as np
import scipy
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import ParameterVector
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import SparsePauliOp, Statevector, state_fidelity, Operator
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.primitives import Estimator
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from scipy.linalg import expm, cosm
import warnings

In [55]:
def get_hamiltonian(nqubits):
    J=1/np.sqrt(2)
    ZZ_tuples = [('ZZ', [i, i+1], J) for i in range(nqubits-1)]
    ZZ_tuples += [('ZZ', [nqubits-1, 0], J)]
    X_tuples = [("X", [i], J) for i in range(nqubits)]
    hamiltonian = SparsePauliOp.from_sparse_list([*ZZ_tuples, *X_tuples], num_qubits=nqubits)
    return hamiltonian.simplify()
    
def ansatz_hamiltonian(nqubits, depth, params):
    circuit = QuantumCircuit(nqubits)
    t = 0
    for i in range(nqubits):
        circuit.h(i)
    for idepth in range(depth):
        for i in range(nqubits):
            if i < (nqubits - 1):
                j = i + 1
            else:
                j = 0
            circuit.rzz(params[t], i, j)
            t += 1
        circuit.barrier()
        for i in range(nqubits):
            if i < (nqubits - 1):
                j = i + 1
            else:
                j = 0    
            circuit.rx(params[t], i)
            t += 1
    return circuit    
    
def ansatz_hea(nqubits, depth, params):
    circuit = QuantumCircuit(nqubits)
    t = 0
    for idepth in range(depth):
        for i in range(nqubits):
            circuit.ry(params[t], i)
            t += 1
        circuit.barrier()
        for i in range(nqubits-1):
            circuit.cx(i, i+1)
    return circuit    
   
def cost_function1(params, nqubits, depth, error_rate):
    hamiltonian =  get_hamiltonian(nqubits)
    circuit =  ansatz_hamiltonian(nqubits, depth,params)
    circuit = circuit.decompose()
    noise_model = NoiseModel()
    error = depolarizing_error(error_rate, 1)
    noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3'])
    error1 = depolarizing_error(error_rate*10, 2)
    noise_model.add_all_qubit_quantum_error(error1,'cx')
    sim_d = AerSimulator(noise_model=noise_model)
    circuit.save_statevector() 
    if error_rate == 0:
        simulator = AerSimulator()
    else:
        simulator = sim_d
        circuit = transpile(circuit, sim_d)
        #noise_result = sim_d.run(circ_noise, shots=1).result()
    result = simulator.run(circuit).result()
    u=result.data(0)['statevector'].data
    expectation = (u.conj().dot(hamiltonian.to_matrix())).dot(u)
    return expectation.real
    
def cost_function2(nqubits, error_rate, params):
    hamiltonian =  get_hamiltonian(nqubits)
    circuit =  ansatz_hea(nqubits, depth,params)
    circuit = circuit.decompose()
    noise_model = NoiseModel()
    error = depolarizing_error(error_rate, 1)
    noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3'])
    error1 = depolarizing_error(error_rate*10, 2)
    noise_model.add_all_qubit_quantum_error(error1,['cx'])
    sim_d = AerSimulator(noise_model=noise_model)
    circuit.save_statevector() 
    if error_rate == 0:
        simulator = AerSimulator()
    else:
        simulator = sim_d
        circuit = transpile(circuit, sim_d)
        #noise_result = sim_d.run(circ_noise, shots=1).result()
    result = simulator.run(circuit).result()
    u=result.data(0)['statevector'].data
    expectation = (u.conj().dot(hamiltonian.to_matrix())).dot(u)
    return expectation.real

def callback(intermediate_result):
    print(
        f"{intermediate_result.fun}"
        )
    #with open('intermediate_values_hva.txt', 'a') as file:
     #   file.write(f'Intermediate values: {intermediate_result.fun}\n')
    


In [73]:
np.random.seed(42)  # Example seed
nqubits = 4
J = 1/np.sqrt(2)
depth = 2
error_rate = 0
parameter = np.array(np.random.random(2*nqubits*depth))
#ansatz_hamiltonian(nqubits, depth, parameter).draw('mpl')
estimate_val_hva = minimize(cost_function1, parameter, args=(nqubits, depth, error_rate), method="BFGS", tol=1e-5, options={'disp': False}, callback=callback)

-1.146546159434933
-2.6028982781680043
-2.848094190781863
-3.206002551624199
-3.3199944173922806
-3.4384045401477366
-3.537362376543939
-3.6054729693003296
-3.6484027412465654
-3.66202303984785
-3.670442776835458
-3.6765937071754573
-3.6807428114669736
-3.6865486487163923
-3.693031336564412
-3.694791992030758
-3.6951775028962928
-3.695327279219989
-3.6954448593592524
-3.695492189546099
-3.6955136490296367
-3.695517967084661
-3.6955181145196816
-3.6955181297534514
-3.6955181300344395


In [70]:
np.random.seed(42)  # Example seed
nqubits = 8
J = 1/np.sqrt(2)
depth = 2
error_rate = 0
parameter = np.array(np.random.random(2*nqubits*depth))
#ansatz_hamiltonian(nqubits, depth, parameter).draw('mpl')
estimate_val_hva = minimize(cost_function1, parameter, args=(nqubits, depth, error_rate), method="BFGS", tol=1e-5, options={'disp': False}, callback=callback)#, args=(nqubits, error_rate)

0.6637236559532338
-2.4083395986041896
-3.3768099134511727
-4.905584489141767
-6.339482878369267
-6.662114046864324
-6.697647890797874
-6.757552744090512
-6.841145704256055
-6.876492802025896
-6.882980241094593
-6.893737594869573
-6.905297203107356
-6.908615324999041
-6.909772586651051
-6.911577177611999
-6.91458536020455
-6.919420107688753
-6.923993924196165
-6.9302207513884975
-6.93905995742495
-6.94112562249331
-6.943583094576102
-6.94516405984821
-6.947217728169778
-6.95026409923488
-6.953054156841418
-6.955821349566385
-6.960541516812028
-6.96690566771797
-6.972040326218838
-6.978267475293993
-6.981266885241748
-6.985860829917177
-6.990175598297801
-6.9946290765312025
-7.000016289099696
-7.002653570596343
-7.006390959710229
-7.00956493617387
-7.011725290335363
-7.01388456579012
-7.015513632073938
-7.0165146249258505
-7.0172969954802795
-7.017991984801805
-7.018473208261488
-7.018741925933037
-7.018921164990434
-7.019082026130388
-7.019275744788116
-7.01935025371885
-7.019433742144

In [74]:
hamiltonian = get_hamiltonian(4)
H_array = hamiltonian.to_matrix()
eval, _ = np.linalg.eigh(H_array)
# print(eval)
emin = eval[0]
emax = eval[-1]
print(emin)

-3.695518130045147


In [71]:
hamiltonian = get_hamiltonian(8)
H_array = hamiltonian.to_matrix()
eval, _ = np.linalg.eigh(H_array)
# print(eval)
emin = eval[0]
emax = eval[-1]
print(emin)

-7.249019570823107


TO BE DONE